In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [2]:
def Convolution(x, filters) :
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)
  x = layers.Conv2D(filters, kernel_size=1, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)
  x = layers.Conv2D(filters, kernel_size=3, padding='same')(x)
  return x

In [3]:
def DenseBlocks(x, num_layers, filters):
  for i in range(num_layers):
    if i == 0:
      layer = Convolution(x, filters)
      outputs = layer
    else :
      layer = Convolution(layer, filters)
      layer = layers.Concatenate(axis=-1)([layer, outputs])
      outputs = layer

  return outputs

In [4]:
def DenseNet(shape) :
  inputs = layers.Input(shape=(shape,shape,1))
  l1 = DenseBlocks(inputs, 5, 12)
  l1 = layers.Conv2D(12, kernel_size=1, padding='same')(l1)
  l1 = layers.AveragePooling2D(pool_size=2)(l1)
  l2 = DenseBlocks(l1, 5, 24)
  l2 = layers.Conv2D(24, kernel_size=1, padding='same')(l2)
  l2 = layers.AveragePooling2D(pool_size=2)(l2)
  l3 = DenseBlocks(l2, 5, 36)
  l3 = layers.Conv2D(36, kernel_size=1, padding='same')(l3)
  l3 = layers.AveragePooling2D(pool_size=2)(l3)
  outputs = layers.GlobalAveragePooling2D()(l3)
  outputs = layers.Dense(10, activation='softmax')(outputs)
  model = tf.keras.Model(inputs, outputs)
  return model

In [5]:
model = DenseNet(28)

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 1)    4           input_1[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 28, 28, 1)    0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 12)   24          re_lu[0][0]                      
_______________________________________________________________________________________

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [8]:
x_train.shape

(60000, 28, 28)

In [9]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [10]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [11]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.7435 - accuracy: 0.1000


[2.74352765083313, 0.10000000149011612]

In [ ]:
model.fit(x_train,y_train, batch_size=128, epochs=50, validation_split=0.2)

In [13]:
model.evaluate(x_test, y_test)

94/94 [==============================] - 1s 9ms/step - loss: 0.3810 - accuracy: 0.8781


[0.38104528188705444, 0.878083348274231]